In [2]:
import pandas as pd
import re
from tqdm import tqdm
import pickle
import copy
tqdm.pandas()

In [3]:
# https://huggingface.co/datasets/heegyu/namuwiki/tree/main
df = pd.read_parquet('data/namuwiki_20210301.parquet')
print(df.head())

        title                                               text  \
0           !                                    #redirect 느낌표\n   
1      !!아앗!!  \n[목차]\n\n'''{{{+1 ！！ああっと！！}}}'''\n\n== 개요 ==\...   
2        “……”  ||<-2><tablebordercolor=#878787><tablealign=ri...   
3           #  [[분류:특수 문자]]\n[include(틀:다른 뜻1, other1=음악에서 사용...   
4  #FairyJoke  [include(틀:링크시 주의, 링크=[[\\#FairyJoke]] 또는 [[#F...   

                                        contributors namespace  
0                                r:hoon12560,namubot            
1  110.46.34.123,kirby10,max0243,218.54.117.149,r...            
2  milkbutter,116.122.29.64,211.54.69.240,180.68....            
3  topenkun,125.186.248.118,benquan1812,180.231.1...            
4  r:db,lotot,swontrdg,58.224.144.55,guylian,39.1...            


In [4]:
df = df[['title', 'text']]
df

,title,text
0,!,#redirect 느낌표\n
1,!!아앗!!,\n[목차]\n\n'''{{{+1 ！！ああっと！！}}}'''\n\n== 개요 ==\...
2,“……”,||<-2><tablebordercolor=#878787><tablealign=ri...
3,#,"[[분류:특수 문자]]\n[include(틀:다른 뜻1, other1=음악에서 사용..."
4,#FairyJoke,"[include(틀:링크시 주의, 링크=[[\\#FairyJoke]] 또는 [[#F..."
...,...,...
867019,광저우 푸리,#redirect 광저우 시티\n
867020,에스시더,#redirect 이스시\n
867021,칭다오 황하이,#redirect 칭다오 FC\n
867022,순혜옹주,[[분류:조선의 후궁]]\n[include(틀:조선의 후궁(환조 ~ 연산군))]\n...


In [9]:
# 테이블이 있는 경우만 골라내기
filtered_df = df[df['text'].str.contains('<table')]
filtered_df

,title,text
2,“……”,||<-2><tablebordercolor=#878787><tablealign=ri...
6,#Fairy_dancing_in_lake,"[include(틀:링크시 주의, 링크=[[\\#Fairy_dancing_in_la..."
14,&(싱글),[include(틀:하마사키 아유미의 싱글)]\n||||||<tablewidth=1...
18,페텔기우스 로마네콩티,"[include(틀:다른 뜻1, other1=항성, rd1=베텔게우스)]\n[inc..."
19,사테라(Re: 제로부터 시작하는 이세계 생활),"[include(틀:토론 합의, 토론주소1=ANullAndFascinatedShoe..."
...,...,...
867012,마른 남자,||<table width=300><table bordercolor=#000><ta...
867014,니나(로얄 크라운),"[[분류:로얄 크라운]]\n[include(틀:상위 문서, top1=로얄 크라운)]..."
867015,혜선옹주,[[분류:조선의 후궁]]\n[include(틀:조선의 후궁(환조 ~ 연산군))]\n...
867016,아즈사 아이자와,||<table width=100%><width=58%><tablebordercol...


In [11]:
# 목차가 들어가 있는게 내용이 어느정도 있는 페이지어서 이 기준으로 필터링함
filtered_df = filtered_df[filtered_df['text'].str.contains('[목차]')]
filtered_df

,title,text
2,“……”,||<-2><tablebordercolor=#878787><tablealign=ri...
6,#Fairy_dancing_in_lake,"[include(틀:링크시 주의, 링크=[[\\#Fairy_dancing_in_la..."
14,&(싱글),[include(틀:하마사키 아유미의 싱글)]\n||||||<tablewidth=1...
18,페텔기우스 로마네콩티,"[include(틀:다른 뜻1, other1=항성, rd1=베텔게우스)]\n[inc..."
19,사테라(Re: 제로부터 시작하는 이세계 생활),"[include(틀:토론 합의, 토론주소1=ANullAndFascinatedShoe..."
...,...,...
867012,마른 남자,||<table width=300><table bordercolor=#000><ta...
867014,니나(로얄 크라운),"[[분류:로얄 크라운]]\n[include(틀:상위 문서, top1=로얄 크라운)]..."
867015,혜선옹주,[[분류:조선의 후궁]]\n[include(틀:조선의 후궁(환조 ~ 연산군))]\n...
867016,아즈사 아이자와,||<table width=100%><width=58%><tablebordercol...


In [12]:
# 가장 앞에 있는 테이블이 정보 테이블인 경우가 대부분이다. 
filtered_df['text'] = filtered_df['text'].apply(lambda x: x.split('[목차]')[0])
filtered_df

<ipython-input-12-8d5787fbdacd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['text'] = filtered_df['text'].apply(lambda x: x.split('[목차]')[0])


,title,text
2,“……”,||<-2><tablebordercolor=#878787><tablealign=ri...
6,#Fairy_dancing_in_lake,"[include(틀:링크시 주의, 링크=[[\\#Fairy_dancing_in_la..."
14,&(싱글),[include(틀:하마사키 아유미의 싱글)]\n||||||<tablewidth=1...
18,페텔기우스 로마네콩티,"[include(틀:다른 뜻1, other1=항성, rd1=베텔게우스)]\n[inc..."
19,사테라(Re: 제로부터 시작하는 이세계 생활),"[include(틀:토론 합의, 토론주소1=ANullAndFascinatedShoe..."
...,...,...
867012,마른 남자,||<table width=300><table bordercolor=#000><ta...
867014,니나(로얄 크라운),"[[분류:로얄 크라운]]\n[include(틀:상위 문서, top1=로얄 크라운)]..."
867015,혜선옹주,[[분류:조선의 후궁]]\n[include(틀:조선의 후궁(환조 ~ 연산군))]\n...
867016,아즈사 아이자와,||<table width=100%><width=58%><tablebordercol...


In [221]:
def get_table_contents(text):
    table_contents = []
    regex = r"\|\|.*\|\|\n"
    matches = re.findall(regex, text)
    if matches:
        for match in matches:
            table_contents.append(match)
    return table_contents

In [223]:
filtered_df['table_contents'] = filtered_df['text'].progress_apply(lambda x: get_table_contents(x))
filtered_df

100%|██████████| 274635/274635 [00:03<00:00, 86791.68it/s]
<ipython-input-223-c12b80ec17d1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['table_contents'] = filtered_df['text'].progress_apply(lambda x: get_table_contents(x))


,title,text,table_contents
2,“……”,||<-2><tablebordercolor=#878787><tablealign=ri...,[||<-2><tablebordercolor=#878787><tablealign=r...
6,#Fairy_dancing_in_lake,"[include(틀:링크시 주의, 링크=[[\\#Fairy_dancing_in_la...",[]
14,&(싱글),[include(틀:하마사키 아유미의 싱글)]\n||||||<tablewidth=1...,[||||||<tablewidth=100%> '''[[하마사키 아유미]]의 싱글''...
18,페텔기우스 로마네콩티,"[include(틀:다른 뜻1, other1=항성, rd1=베텔게우스)]\n[inc...",[||<tablealign=center><bgcolor=#ffffff> [[파일:페...
19,사테라(Re: 제로부터 시작하는 이세계 생활),"[include(틀:토론 합의, 토론주소1=ANullAndFascinatedShoe...",[||<-2><bgcolor=#414a4c> {{{#white 애니메이션}}}[* ...
...,...,...,...
867012,마른 남자,||<table width=300><table bordercolor=#000><ta...,[||<table width=300><table bordercolor=#000><t...
867014,니나(로얄 크라운),"[[분류:로얄 크라운]]\n[include(틀:상위 문서, top1=로얄 크라운)]...",[||<tablealign=center><tablewidth=70%><-2><bgc...
867015,혜선옹주,[[분류:조선의 후궁]]\n[include(틀:조선의 후궁(환조 ~ 연산군))]\n...,[||<-2><tablealign=right><tablewidth=400><tabl...
867016,아즈사 아이자와,||<table width=100%><width=58%><tablebordercol...,[||<table width=100%><width=58%><tableborderco...


In [227]:
def parse_table_contents(table_contents):
    parsed_table_contents = []
    for content in table_contents:
        content = content.strip().strip('||')
        if '<table' in content:
            continue
        splited_content = content.split('||')
        if len(splited_content) == 1:
            continue
        if len(splited_content) > 2:
            splited_content = splited_content[-2:] 
        predicate, objects = splited_content
        parsed_table_contents.append({'predicate': predicate, 'objects': objects})
    return parsed_table_contents

In [230]:
filtered_df['parsed_table_contents'] = filtered_df['table_contents'].progress_apply(lambda x: parse_table_contents(x))
filtered_df

100%|██████████| 274635/274635 [00:27<00:00, 9872.27it/s] 
<ipython-input-230-a5ab3ba10b00>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['parsed_table_contents'] = filtered_df['table_contents'].progress_apply(lambda x: parse_table_contents(x))


,title,text,table_contents,parsed_table_contents
2,“……”,||<-2><tablebordercolor=#878787><tablealign=ri...,[||<-2><tablebordercolor=#878787><tablealign=r...,"[{'predicate': ' '''코너명''' ', 'objects': ' “……..."
6,#Fairy_dancing_in_lake,"[include(틀:링크시 주의, 링크=[[\\#Fairy_dancing_in_la...",[],[]
14,&(싱글),[include(틀:하마사키 아유미의 싱글)]\n||||||<tablewidth=1...,[||||||<tablewidth=100%> '''[[하마사키 아유미]]의 싱글''...,"[{'predicate': '<width=40%> '''&''' (2003) ', ..."
18,페텔기우스 로마네콩티,"[include(틀:다른 뜻1, other1=항성, rd1=베텔게우스)]\n[inc...",[||<tablealign=center><bgcolor=#ffffff> [[파일:페...,[{'predicate': '<bgcolor=#007f66> {{{#ffffff '...
19,사테라(Re: 제로부터 시작하는 이세계 생활),"[include(틀:토론 합의, 토론주소1=ANullAndFascinatedShoe...",[||<-2><bgcolor=#414a4c> {{{#white 애니메이션}}}[* ...,[]
...,...,...,...,...
867012,마른 남자,||<table width=300><table bordercolor=#000><ta...,[||<table width=300><table bordercolor=#000><t...,[]
867014,니나(로얄 크라운),"[[분류:로얄 크라운]]\n[include(틀:상위 문서, top1=로얄 크라운)]...",[||<tablealign=center><tablewidth=70%><-2><bgc...,"[{'predicate': ' {{{#000,#fff '''클래스'''}}} ', ..."
867015,혜선옹주,[[분류:조선의 후궁]]\n[include(틀:조선의 후궁(환조 ~ 연산군))]\n...,[||<-2><tablealign=right><tablewidth=400><tabl...,[{'predicate': '<width=50> '''{{{#gold 본명}}}''...
867016,아즈사 아이자와,||<table width=100%><width=58%><tablebordercol...,[||<table width=100%><width=58%><tableborderco...,[{'predicate': '<rowbgcolor=#ffff99> [[전생했더니 슬...


In [232]:
# 파싱된 결과가 0 이상인 것만 남긴다. 
cnt_df = filtered_df[['title', 'parsed_table_contents']]
cnt_df['parsed_table_contents'] = cnt_df['parsed_table_contents'].apply(lambda x: len(x))
cnt_df

<ipython-input-232-f18afb6f8856>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cnt_df['parsed_table_contents'] = cnt_df['parsed_table_contents'].apply(lambda x: len(x))


,title,parsed_table_contents
2,“……”,5
6,#Fairy_dancing_in_lake,0
14,&(싱글),1
18,페텔기우스 로마네콩티,1
19,사테라(Re: 제로부터 시작하는 이세계 생활),0
...,...,...
867012,마른 남자,0
867014,니나(로얄 크라운),9
867015,혜선옹주,5
867016,아즈사 아이자와,7


In [233]:
filtered_df = filtered_df[filtered_df['title'].isin(cnt_df[cnt_df['parsed_table_contents']!=0]['title'].unique())]
filtered_df

,title,text,table_contents,parsed_table_contents
2,“……”,||<-2><tablebordercolor=#878787><tablealign=ri...,[||<-2><tablebordercolor=#878787><tablealign=r...,"[{'predicate': ' '''코너명''' ', 'objects': ' “……..."
14,&(싱글),[include(틀:하마사키 아유미의 싱글)]\n||||||<tablewidth=1...,[||||||<tablewidth=100%> '''[[하마사키 아유미]]의 싱글''...,"[{'predicate': '<width=40%> '''&''' (2003) ', ..."
18,페텔기우스 로마네콩티,"[include(틀:다른 뜻1, other1=항성, rd1=베텔게우스)]\n[inc...",[||<tablealign=center><bgcolor=#ffffff> [[파일:페...,[{'predicate': '<bgcolor=#007f66> {{{#ffffff '...
21,레굴루스 코르니아스,[include(틀:대죄주교(Re: 제로부터 시작하는 이세계 생활))]\n\n||<...,[||<bgcolor=#c0c0c0> {{{#ffffff '''라이트 노벨 설정''...,[{'predicate': '<bgcolor=#c0c0c0> {{{#ffffff '...
22,에키드나(Re: 제로부터 시작하는 이세계 생활),[include(틀:회원수정)]\n[include(틀:마녀(Re: 제로부터 시작하는...,[||<width=57%><bgcolor=#ffffff> [[파일:에키드나 라노벨 ...,[{'predicate': '<width=57%><bgcolor=#ffffff> [...
...,...,...,...,...
867005,불량배들(리틀 나이트메어 시리즈),||<-2><table width=300><table bordercolor=#000...,[||<-2><table width=300><table bordercolor=#00...,[{'predicate': '<width=50%> [[파일:Male_Bully.pn...
867014,니나(로얄 크라운),"[[분류:로얄 크라운]]\n[include(틀:상위 문서, top1=로얄 크라운)]...",[||<tablealign=center><tablewidth=70%><-2><bgc...,"[{'predicate': ' {{{#000,#fff '''클래스'''}}} ', ..."
867015,혜선옹주,[[분류:조선의 후궁]]\n[include(틀:조선의 후궁(환조 ~ 연산군))]\n...,[||<-2><tablealign=right><tablewidth=400><tabl...,[{'predicate': '<width=50> '''{{{#gold 본명}}}''...
867016,아즈사 아이자와,||<table width=100%><width=58%><tablebordercol...,[||<table width=100%><width=58%><tableborderco...,[{'predicate': '<rowbgcolor=#ffff99> [[전생했더니 슬...


In [ ]:
def refine_triple(triple):
    refined_predicate = re.findall(r"'''(.*?)'''", triple['predicate'])
    triple_list = []
    if not refined_predicate:
        return None
    else:
        refined_predicate = refined_predicate[0]
        refined_predicate = refined_predicate.replace('[br]', '')
        refined_predicate = refined_predicate.split('|')[0].strip('[[')
    if '[[' in triple['objects'] and ']]' not in triple['objects']:
        triple['objects'] = triple['objects'] + ']]'
    refined_objects = re.findall(r'\[\[(.*?)\]\]', triple['objects'])
    for obj in refined_objects:
        triple_list.append({'predicate': refined_predicate, 'objects': obj})
    return triple_list

def get_refine_triple(triples):
    triple_list = []
    for triple in triples:
        refined_triple = refine_triple(triple)
        if refined_triple:
            for rt in refined_triple:
                if rt['objects']:
                    triple_list.append(rt)
    if len(triple_list) == 0:
        return None
    return triple_list

def find_korean_text(x):
    korean_pattern = re.compile(r"[가-힣]+")
    korean_matches = korean_pattern.findall(x)
    if len(korean_matches) == 0:
        return None
    return korean_matches[0]

In [ ]:
filtered_df['refined_triple'] = filtered_df['parsed_table_contents'].progress_apply(lambda x: get_refine_triple(x))
triple_df = filtered_df[['title', 'refined_triple']].dropna()
triple_df = triple_df.explode('refined_triple')
triple_df['predicate'] = triple_df['refined_triple'].apply(lambda x: x['predicate'])
triple_df['object'] = triple_df['refined_triple'].apply(lambda x: x['objects'])
triple_df = triple_df[['title', 'predicate', 'object']]
triple_df['predicate'] = triple_df['predicate'].progress_apply(lambda x: find_korean_text(x))
triple_df = triple_df.dropna()

In [ ]:
predicate_cnt_df = triple_df.groupby('predicate')['object'].count().reset_index()
filtered_predicates = predicate_cnt_df[predicate_cnt_df['object']>=200]['predicate'].unique()
filtered_triple_df = triple_df[triple_df['predicate'].isin(filtered_predicates)]
filtered_triple_df

In [ ]:
filtered_triple_object_refined = copy.deepcopy(filtered_triple_df)
filtered_triple_object_refined['object'] = filtered_triple_object_refined['object'].apply(lambda x: x.split('|')[0].strip())
# 오류가 많아보이는 predicate 제외
exclude_predicates = ['가고시마', '가족관계', '닛포', '인간관계', '부모', '가족', '형제자매']
excloude_objects - ['아버지', '어머니', '누나', '형', '동생' '동생', '남동생']
filtered_triple_object_refined = filtered_triple_object_refined[~filtered_triple_object_refined['predicate'].isin(exclude_predicates)]
filtered_triple_object_refined = filtered_triple_object_refined[~filtered_triple_object_refined['object'].isin(exclude_objects)
filtered_triple_object_refined['title'] = filtered_triple_object_refined['title'].apply(lambda x: f'https://namu.wiki/w/{x}')
filtered_triple_object_refined['object'] = filtered_triple_object_refined['object'].apply(lambda x: f'https://namu.wiki/w/{x}')
filtered_triple_object_refined['predicate'] = filtered_triple_object_refined['predicate'].apply(lambda x: f'predicate/{x}')
filtered_triple_object_refined.columns = ['subject', 'predicate', 'object']
filtered_triple_object_refined

In [ ]:
# graphDB에 올리기 위해 rdf 규칙에 맞게 수정. 각종 오류 일으키는 원인 제거

triple = copy.deepcopy(filtered_triple_object_refined)
triple['subject'] = triple['subject'].apply(lambda x:f"<{x.replace('<', '').replace('>', '')}>")
triple['predicate'] = triple['predicate'].apply(lambda x:f"<https://{x.replace('<', '').replace('>', '')}>")
triple['object'] = triple['object'].apply(lambda x:f'<{x}>')

triple['subject'] = triple['subject'].apply(lambda x: '_'.join(x.split()))
triple['predicate'] = triple['predicate'].apply(lambda x: '_'.join(x.split()))
triple['object'] = triple['object'].apply(lambda x: '_'.join(x.split()))


triple = triple[~triple['subject'].str.contains('%|#|\^|\{|\}|`|\|')]
triple = triple[~triple['object'].str.contains('%|#|\^|\{|\}|`|\|')]
triple = triple[~triple['object'].str.contains('.jpg')]
triple = triple[~triple['object'].str.contains('.svg')]
triple = triple[~triple['object'].str.contains('.png')]
triple = triple[~triple['object'].str.contains('파일:')]
triple['subject'] = triple['subject'].apply(lambda x: x.replace('\\', '').replace('"', '').replace(']', '').replace('[', ''))
triple['object'] = triple['object'].apply(lambda x: x.replace('\\', '').replace('"', '').replace(']', '').replace('[', ''))
triple = triple[~triple['subject'].str.contains('Réalta_Nua')]
triple = triple[~triple['subject'].str.contains('/Fate/')]
triple['end'] = '.'